In [10]:
import pandas as pd
import re

In [5]:
class Info(object):
    def __init__(self, start_str, end_str, delaystart=0):
        self.start = False
        self.saved = False
        self.start_str = start_str
        self.end_str = end_str
        self.delay_start = delaystart
        
    def parse(self, line):
        if self.start_str in line[:len(self.start_str)]:
            if self.saved and not self.start:
                # if info has been saved, this could be a false start-signal
                return
                
            self.start = True
            
        if self.start and self.delay_start > 0:
            self.delay_start -= 1
            # delay the start
            return

        if self.end_str == line[:len(self.end_str)]:
            self.start = False

        if self.start:
            self.add(line)
            self.saved = True
    
    def add(self, line):
        pass
    
    
    def out(self):
        pass

In [6]:
class Timing(Info):
    def __init__(self):
        super().__init__('*****Chrono*****', 'ChronoStatSvc    ', delaystart=2)
        
        self.tool_list_l = []
        self.total_l = []
        self.avg_l = []
        self.min_l = []
        self.max_l = []
        self.evts_1 = []
        
    
    @classmethod
    def get_time_scale(self, unit):
        if "us" in unit:
            time_scale = 1e-6
        elif 'ms' in unit:
            time_scale = 1e-3
        elif 'min' in unit:
            time_scale = 60
        elif 's' in unit:
            time_scale = 1
        else:
            time_scale = -1
        return time_scale
    
    def add(self, line):
        has_avg = "Ave/Min/Max" in line
        items = line[:-1].split()
        
        tool_name = items[0]
        # don't perform replacement on the toolName
        new_str = " ".join(items[1:])
        new_str = new_str.replace('=', ' = ')
        new_str = new_str.replace('/', ' / ')
        new_str = new_str.replace('(', ' ( ')
        new_str = new_str.replace(')', ' ) ')
        new_str = new_str.replace('[', ' [ ')
        new_str = new_str.replace('+-', ' +- ')
        items = [items[0]] + new_str.split()
        
        
        try:
            time_unit = items[9]
        except IndexError:
            print(line)
            return
            
        time_scale = self.get_time_scale(items[9])
        if time_scale < 0:
            print("Time error0:", items)
        
        try:
            total_ = float(items[7])*time_scale
        except:
            total_ = 0
            print("Total ERROR:", items)
            
        
        evts_input_index = 12
        if has_avg:
            evts_input_index = 29
            
            try:
                time_unit = items[26]
            except IndexError:
                print("Time ERROR1:", items)
                return
                
            time_scale = self.get_time_scale(time_unit)
            if time_scale < 0:
                print("Time ERROR2", items)
            
            avg_ = float(items[16])*time_scale
            min_ = float(items[22])*time_scale
            max_ = float(items[24])*time_scale
        else:
            avg_ = min_ = max_ = -999
        
        try:
            evts_ = int(items[evts_input_index])
        except:
            evts_ = 0
            print("Evt ERROR", items)
            
        
        self.tool_list_l.append(tool_name)
        self.total_l.append(total_)
        self.avg_l.append(avg_)
        self.min_l.append(min_)
        self.max_l.append(max_)
        self.evts_1.append(evts_)
        
    def to_str(self):
        print(len(self.tool_list_l), len(self.total_l))
        
        
    def out(self):
        data = {
            "Tool":  self.tool_list_l,
            "Total": self.total_l,
            "Ave":   self.avg_l,
            "Min":   self.min_l,
            "Max":   self.max_l,
            "Nevt":  self.evts_1
        }
        df = pd.DataFrame(data)
        return df

In [7]:
class InDetTrackParticles(Info):
    def __init__(self):
        super().__init__('InDetTrackParticles    ', 'InDetForwardTrackParticles', delaystart=0)
        self.out_str = ""
        
    def add(self, line):
        self.out_str += line
        
    def out(self):
        return self.out_str

In [12]:
class Match(object):
    def __init__(self, pattern):
        self.pp = re.compile(pattern)
        self.out_str = ""
        
    def parse(self, line):
        if self.pp.match(line):
            self.out_str += line
        
    def out(self):
        return self.out_str

In [15]:
inDetTrackParticlesInfo = Match('InDetTrackParticles ')

vtxFinder = Match('InDetPriVxFinder')
timing_info = Timing()
handle = open('../data/log.add_permonsd.500evts')
for line in handle:
    timing_info.parse(line)
    inDetTrackParticlesInfo.parse(line)
    vtxFinder.parse(line)

handle.close()

*****Chrono*****                                     INFO ****************************************************************************************************



In [8]:
df = timing_info.out()
df.sort_values(by=['Total'], ascending=False).head(15)

,Tool,Total,Ave,Min,Max,Nevt
3149,AthMasterSeq:Execute,4116.0,8.2200,1.200000,43.560,500
3148,AthAlgSeq:Execute,4116.0,8.2200,1.200000,43.560,500
3147,InDetSiSpTrackFinder:Execute,1338.0,2.6700,0.154200,20.100,500
3146,InDetAmbiguitySolver:Execute,828.0,1.6620,0.181200,9.900,500
3145,InDetExtensionProcessor:Execute,574.8,1.1520,0.205800,4.338,500
3144,StreamESD:Execute,262.0,0.5240,0.095000,2.430,500
3143,commitOutput,199.0,0.1980,0.000999,1.110,1002
3142,InDetTRT_SeededTrackFinder:Execute,150.0,0.3000,0.004000,2.140,500
3140,InDetTRT_Extension:Execute,96.2,0.1920,0.031000,0.681,500
3141,InDetTRT_RIO_Maker:Execute,92.3,0.1850,0.079000,0.255,500


In [16]:
print(inDetTrackParticlesInfo.out())

InDetTrackParticles                                  INFO  --- InDetParticleCreation Summary: 
InDetTrackParticles                                  INFO  --- Input  TrackCollection        : "Tracks" with 320209 Tracks (640.418 per event).
InDetTrackParticles                                  INFO  --- Output TrackParticleContainer : "InDetTrackParticles" with 320209.000 TrackParticles (640.418 per event).
InDetTrackParticles                                  INFO  ---  /Track in   ----->  BLayer  Pixel   SCT    TRT    All (P+S+T)
InDetTrackParticles                                  INFO  ---  Hits                0.805  3.538  7.373  21.009  31.920
InDetTrackParticles                                  INFO  ---  Shared Hits         0.015  0.020  0.210           0.231
InDetTrackParticles                                  INFO  ---  Spoiled/Tube Hits          0.067  0.026   9.017   9.110
InDetTrackParticles                                  INFO  ---  Outliers            0.009  0.027  0.033   

In [14]:
print(vtxFinder.out())

InDetPriVxFinder                                     INFO Retrieved tool VertexFinderTool = PublicToolHandle('InDet::InDetIterativePriVxFinderTool/InDetIterativePriVxFinderTool')
InDetPriVxFinder                                     INFO Retrieved tool VertexCollectionSortingTool = PublicToolHandle('Trk::VertexCollectionSortingTool/InDetVertexCollectionSortingTool')
InDetPriVxFinder                                     INFO Initialization successful
InDetPriVxFinder                                     INFO Summary from Primary Vertex Finder (InnerDetector/InDetRecAlgs/InDetPriVxFinder)
InDetPriVxFinder                                     INFO === 10264 vertices recoed in 500 events (excluding dummy).
InDetPriVxFinder                                     INFO === 20.528 vertices per event (excluding dummy).
InDetPriVxFinder:Start                               INFO Time User   : Tot=    0 [us]                                             #=  1
InDetPriVxFinder:Stop                           